In [1]:
# !pip install transformers
# !pip install accelerate
# !pip install twine
# !pip install datasets
# !pip install tyro

In [2]:
!nvidia-smi

Wed Mar 20 11:37:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB            On | 00000000:3B:00.0 Off |                  Off |
| N/A   31C    P0               24W / 250W|      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# !pip install wandb
# !pip install trl
# !pip install pandas
# !pip install datasets
# !pip install nltk -U

In [4]:
import torch
from tqdm import tqdm
import pandas as pd
import wandb
import os

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [5]:
config = PPOConfig(
    model_name    = "openai-community/gpt2",
    learning_rate = 1.41e-5,
    ## log_with      = "wandb",
)

sent_kwargs = {
         "return_all_scores": True, 
         "function_to_apply": "none", 
         "batch_size": 16
}
print(config)

PPOConfig(exp_name='ipykernel_launcher', seed=0, log_with=None, task_name=None, model_name='openai-community/gpt2', query_dataset='imdb', reward_model='sentiment-analysis:lvwerra/distilbert-imdb', remove_unused_columns=True, tracker_kwargs={}, accelerator_kwargs={}, project_kwargs={}, tracker_project_name='trl', push_to_hub_if_best_kwargs={}, steps=20000, learning_rate=1.41e-05, adap_kl_ctrl=True, init_kl_coef=0.2, kl_penalty='kl', target=6, horizon=10000, gamma=1, lam=0.95, cliprange=0.2, cliprange_value=0.2, vf_coef=0.1, batch_size=128, forward_batch_size=None, mini_batch_size=128, gradient_accumulation_steps=1, world_size=None, ppo_epochs=4, max_grad_norm=None, optimize_cuda_cache=None, optimize_device_cache=False, early_stopping=False, target_kl=1, compare_steps=1, ratio_threshold=10.0, use_score_scaling=False, use_score_norm=False, score_clip=None, whiten_rewards=False, is_encoder_decoder=None, is_peft_model=None, backward_batch_size=128, global_backward_batch_size=None, global_ba

In [6]:
## wandb.init()

wandb.init(mode="disabled") 
os.environ['WANDB_DISABLED'] = 'true'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [7]:
dataset_name="ag_news"

In [8]:
ds = load_dataset(dataset_name, split = "train[:100000]")

In [9]:
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 100000
})

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [11]:
tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token
# print(tokenizer)

In [12]:
def tokenize( sample ):
    sample["input_ids"] = tokenizer.encode( sample["text"]    )[: 100]
    sample["query"]     = tokenizer.decode( sample["input_ids"] )
    return sample

#print(tokenize)
ds = ds.map(tokenize, batched=False)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
print(ds)

Dataset({
    features: ['text', 'label', 'input_ids', 'query'],
    num_rows: 100000
})


In [14]:
def build_dataset(
         config, 
         dataset_name="ag_news", 
         input_min_text_length=2, 
         input_max_text_length=8
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    # load with datasets
    
    ds = load_dataset(dataset_name, split="train[:100000]")
    
#     ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["text"]) > 100, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode( sample["text"]    )[: input_size()]
        sample["query"]     = tokenizer.decode( sample["input_ids"] )
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [15]:
dataset = build_dataset(config)

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/99939 [00:00<?, ? examples/s]

In [16]:
dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'query'],
    num_rows: 99939
})

In [17]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [18]:
model     = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

In [19]:
ppo_trainer = PPOTrainer(
                 config, 
                 model, 
                 ref_model, 
                 tokenizer, 
                 dataset=dataset, 
                 data_collator=collator
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
device = ppo_trainer.accelerator.device
device

device(type='cuda')

In [21]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

device

0

In [22]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="wesleyacheng/news-topic-classification-with-bert")

In [23]:
generation_kwargs = {
    "min_length":     -1,
    "top_k":         0.0,
    "top_p":         1.0,
    "do_sample":    True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [24]:
output_min_length     = 4
output_max_length     = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [25]:
ppo_trainer.config.steps

20000

In [26]:
# import os
# from transformers import GPT2LMHeadModel

# def save_checkpoint(model, filepath):
#     model.save_pretrained(filepath)
#     print("Model checkpoint saved successfully.")

# # Load Model Checkpoint
# def load_checkpoint(filepath):
#     if os.path.isdir(filepath):
#         model = GPT2LMHeadModel.from_pretrained(filepath)
#         print("Model checkpoint loaded successfully.")
#     else:
#         raise FileNotFoundError(f"Checkpoint directory not found at {filepath}")
#     return model



In [27]:
# import torch

# def gpu_memory_almost_full(threshold=0.9):
#     """
#     Check if GPU memory is almost full.
    
#     Args:
#         threshold (float): Threshold percentage for GPU memory usage.
#             If the current GPU memory usage exceeds this threshold, 
#             consider the GPU memory almost full. Default is 0.9 (90%).
    
#     Returns:
#         bool: True if GPU memory is almost full, False otherwise.
#     """
#     allocated_bytes = torch.cuda.memory_allocated()
#     total_bytes = torch.cuda.get_device_properties(0).total_memory  # assuming GPU 0
#     utilization = allocated_bytes / total_bytes
#     return utilization >= threshold

In [28]:
# checkpoint_dir = '/home/vemuri8/gpt2_checkpoint'
# if os.path.exists(checkpoint_dir):
#     model = load_checkpoint(checkpoint_dir)
# else:
#     # Replace "model_name" with the actual model name or identifier
#     model = GPT2LMHeadModel.from_pretrained("gpt2_checkpoint")

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = pipe(texts, **sent_kwargs)
    rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(
                     query_tensors, 
                     response_tensors, 
                     rewards
    )
#     ppo_trainer.log_stats(stats, batch, rewards)
#     if gpu_memory_almost_full():
#     save_checkpoint(model, 'gpt2_checkpoint')
#         break



0it [00:00, ?it/s]

0


/home/vemuri8/.local/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
1it [00:19, 19.67s/it]

1


2it [00:40, 20.23s/it]

...

720


721it [3:54:09, 18.77s/it]

721


722it [3:54:28, 18.88s/it]

722


In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
bs                 = 16
game_data          = dict()

In [ ]:
game_data 

In [ ]:
dataset.set_format("pandas")

In [ ]:
df_batch           = dataset[:].sample(bs)
df_batch 

In [ ]:
game_data["query"] = df_batch["query"].tolist()
query_tensors      = df_batch["input_ids"].tolist()

In [ ]:
response_tensors_ref, response_tensors = [], []

In [ ]:
gen_kwargs = {
         "min_length":   -1, 
         "top_k":       0.0, 
         "top_p":       1.0, 
         "do_sample":  True, 
         "pad_token_id": tokenizer.eos_token_id
}

In [ ]:
for i in range(bs):
    gen_len = output_length_sampler()
    
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    
    
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

In [ ]:
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

In [ ]:
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in pipe(texts, **sent_kwargs)]

In [ ]:
texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in pipe(texts, **sent_kwargs)]

In [ ]:
df_results = pd.DataFrame(game_data)
df_results

In [ ]:
!nvidia-smi